In [1]:
from tkinter import *
from PIL import Image,ImageTk
from tkcalendar import DateEntry
from tkinter import ttk
import pyodbc as odbc
from tkinter import messagebox
from datetime import date
import datetime
from datetime import datetime
from datetime import datetime
from datetime import datetime

import os
import tempfile
import webbrowser
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import mm
from tkinter import filedialog
import tkinter.font as tkFont
from datetime import datetime

from tkinter import messagebox
import customtkinter as ctk
import functools
from tkcalendar import DateEntry
from tkinter import ttk
from tkinter import *
import tkinter as tk
import pymysql
from tkinter import messagebox,filedialog
from flask import Flask, session, redirect, url_for
import pandas as pd
import tabula
import pdfplumber
import os
import html5lib
import bs4
import numpy as np
import re
import pyodbc
import sqlite3
import time
from tkinter import Frame, Label, Button, Entry, Checkbutton, IntVar, LabelFrame, messagebox, StringVar, PhotoImage
from tkinter.constants import X, BOTH, LEFT, GROOVE, RAISED, W
from tkcalendar import DateEntry
import csv
import pypyodbc as odbc
from decimal import Decimal, InvalidOperation
from PIL import Image, ImageTk
from collections import Counter
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import Toplevel
import matplotlib.pyplot as plt
from datetime import datetime




conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=SAMEER\SQLEXPRESS;'
    r'DATABASE=version1;'
    r'Trusted_Connection=yes;'
        )

# Setup
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

app = ctk.CTk()
app.title("ElectroSales - Dashboard")
app.geometry("1350x700+0+0")

import datetime

# Get current date and time
now = datetime.datetime.now()
formatted_datetime = now.strftime("%A, %d %B %Y | %I:%M %p")

open_windows = {}

# Sidebar
sidebar = ctk.CTkFrame(app, width=200, corner_radius=0, fg_color="#1E2A38")
sidebar.pack(side="left", fill="y")

ctk.CTkLabel(sidebar, text="Menu", font=("Arial", 18, "bold"), text_color="white").pack(pady=(20, 10))

# Dashboard button (highlighted by default)
dashboard_btn = ctk.CTkButton(sidebar, text="Dashboard", width=180, fg_color="#0F62FE", corner_radius=8)
dashboard_btn.pack(pady=5)

# Inventory
inventory_btn = ctk.CTkButton(sidebar, text="Inventory", width=180, fg_color="#2C3E50", corner_radius=8)
inventory_btn.pack(pady=5)



# --- style constants (tweak to match your dashboard) ---
DASH_BG = "#1f2f44"           # deep background for popup container
OUTER_BG = "#0f233d"          # slightly darker outer to simulate shadow/depth
BTN_BG = "#2C3E50"            # billing anchor button bg
POPUP_BTN_BG = "#0f4d7d"      # popup button base
BTN_FG = "white"
FONT = ("Segoe UI", 14)
CORNER_RADIUS = 12
PADDING = 6

def auto_fill_customer_data(event=None):
    global billing_table  # <- this makes billing_table accessible

    input_name = Customer_Name.get().strip()
    input_bill = Bill_Number_entry.get().strip()

    if not input_name and not input_bill:
        return

    cursor = conn.cursor()

    if input_name:
        query = "SELECT * FROM invoices WHERE customer_name LIKE ?"
        param = (input_name + '%',)
    else:
        query = "SELECT * FROM invoices WHERE bill_no LIKE ?"
        param = (input_bill + '%',)

    cursor.execute(query, param)
    customer_row = cursor.fetchone()

    if customer_row:
        bill_no = customer_row[0]
        customer_name = customer_row[1]
        contact = customer_row[2]
        shipping_address = customer_row[3]
        gstin = customer_row[4]
        bill_date = customer_row[12]
        bill_time = customer_row[13]
        status = customer_row[14]

        Customer_Name.delete(0, END)
        Customer_Name.insert(0, customer_name)

        Bill_Number_entry.delete(0, END)
        Bill_Number_entry.insert(0, bill_no)

        Bill_Date_date.set_date(bill_date)
        Delivery_Date_entry.set_date(bill_date)

        Shipping_Address_entry.delete(0, END)
        Shipping_Address_entry.insert(0, shipping_address)

        Status_entry_combobox.set(status)

        Time_entry.delete(0, END)
        Time_entry.insert(0, bill_time)

        Phone_no_entry.delete(0, END)
        Phone_no_entry.insert(0, contact)

        # 🔄 Clear old rows in billing_table
        for item in billing_table.get_children():
            billing_table.delete(item)

        # 🔄 Fetch item rows
        cursor.execute("SELECT * FROM invoice_items WHERE bill_no = ?", (bill_no,))
        item_rows = cursor.fetchall()

        for row in item_rows:
            (
                item_id, bill_no, material_id, description, rating, cat_no, make,
                quantity, net_qty, list_price, discount, disc_unit_price,
                disc_gross_price, lp_gross_price
            ) = row

            billing_table.insert('', END, values=(
                '',  # ☐ Checkbox column placeholder
                bill_no,description, rating, cat_no, make, quantity, net_qty,
                list_price, discount, disc_unit_price, disc_gross_price, lp_gross_price
            ))

    cursor.close()


   

   

def Billing_setup(anchor_widget):
    billing_win = Toplevel(app, bd=0, highlightthickness=0)
    billing_win.overrideredirect(True)  # Remove title bar

    # Position next to anchor
    billing_win.update_idletasks()
    x = anchor_widget.winfo_rootx() + anchor_widget.winfo_width()
    y = anchor_widget.winfo_rooty()
    billing_win.geometry(f"250x240+{x}+{y}")
    billing_win.resizable(False, False)

    # Close when focus lost
    billing_win.bind("<FocusOut>", lambda e: billing_win.destroy())
    billing_win.focus_force()

    # Outer frame to simulate subtle shadow / depth
    outer = ctk.CTkFrame(billing_win, corner_radius=CORNER_RADIUS + 2, fg_color=OUTER_BG)
    outer.pack(fill="both", expand=True, padx=4, pady=4)

    # Inner container with padding and dark background
    container = ctk.CTkFrame(outer, corner_radius=CORNER_RADIUS, fg_color=DASH_BG)
    container.pack(fill="both", expand=True, padx=PADDING, pady=PADDING)

    # Popup buttons styled like dashboard
    btn1 = ctk.CTkButton(
        container,
        text="Bill Entry",
        font=FONT,
        width=220,
        corner_radius=CORNER_RADIUS,
        fg_color=POPUP_BTN_BG,
        text_color=BTN_FG,
        hover=True,
        command= launch_billing_ui
    )
    btn1.pack(fill="x", pady=(0, 8))

    btn2 = ctk.CTkButton(
        container,
        text="Bill Browser",
        font=FONT,
        width=220,
        corner_radius=CORNER_RADIUS,
        fg_color=POPUP_BTN_BG,
        text_color=BTN_FG,
        hover=True,
        command=Browser
    )
    btn2.pack(fill="x", pady=(0, 8))

    btn3 = ctk.CTkButton(
        container,
        text="Payment History",
        font=FONT,
        width=220,
        corner_radius=CORNER_RADIUS,
        fg_color=POPUP_BTN_BG,
        text_color=BTN_FG,
        hover=True,
        command=payment_history
    )
    btn3.pack(fill="x")

# Billing Button with consistent dashboard styling
billing_btn = ctk.CTkButton(
    sidebar,
    text="Billing",
    width=180,
    height=40,
    fg_color=BTN_BG,
    text_color=BTN_FG,
    corner_radius=CORNER_RADIUS,
    font=FONT,
    hover=True,
    command=lambda: Billing_setup(billing_btn)
)
billing_btn.pack(pady=5)

def launch_billing_ui():
    pass
    
def Browser():
        global back_image, total_label
        global Customer_Name, Bill_Number_entry, Bill_Date_date, Delivery_Date_entry
        global Shipping_Address_entry, Status_entry_combobox, Time_entry, Phone_no_entry
        global Item_Description_Entry, Rating_Entry, CAT_NO_Entry, HSN_NO_Entry
        global QTY_Entry, UNIT_Entry, ENCLOSURE_FABRICATION_Entry, EARTHING_Entry
        global FABRICATION_Entry, WIRING_Entry, WIRE_ACCESSORIES_Entry, LABOUR_Entry
        global TRANSPORTATION_Entry, REMARKS_Entry, tree
    
        
        billing_frame = Frame(app, width=2050, height=1100, bg="white")
        billing_frame.place(x=0, y=0)
        
        
        # Header
        heading_label = Label(billing_frame, text="Billing Section", 
                             font=("times new roman", 16, "bold"), bg="#0f4d7d", fg="white")
        heading_label.place(x=0, y=0, relwidth=1)
        
        back_image = PhotoImage(file="left_arrow.png")
        back_button = Button(billing_frame, image=back_image, bd=0, cursor="hand2", 
                            bg="white", command=lambda: billing_frame.place_forget())
        back_button.place(x=10, y=30, height=20, width=20)
        
        # Invoice Details Frame
        details_frame = Frame(billing_frame, bg="white")
        details_frame.place(x=20, y=60, width=1740, height=300)
        
        
        # Invoice Fields
        Label(details_frame, text="Customer Name:", font=("times new roman", 15), bg="white").grid(row=0, column=0, padx=10, pady=5)
        Customer_Name = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Customer_Name.grid(row=0, column=1, padx=10, pady=5)
        Customer_Name.bind('<KeyRelease>', auto_fill_customer_data)
    
       
    
        Label(details_frame, text="Bill Date:", font=("times new roman", 15), bg="white").grid(row=0, column=2, padx=10, pady=5)
        Bill_Date = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Bill_Date.grid(row=0, column=3, padx=10, pady=5)
        Bill_Date_date=DateEntry(details_frame,width=18,font=("times new roman",15),state="readonly",date_pattern="dd/mm/yyyy")
        Bill_Date_date.grid(row=0,column=3,padx=10,pady=5)
    
        
        Label(details_frame, text="Bill_Number:", font=("times new roman", 15), bg="white").grid(row=0, column=4, padx=10, pady=5)
        Bill_Number_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Bill_Number_entry.grid(row=0, column=5, padx=10, pady=5)
        Bill_Number_entry.bind('<KeyRelease>', auto_fill_customer_data)
    
        search_button = Button(details_frame, text="Search", font=("times new roman", 15), bg="#4CAF50", fg="white", width=10, command=lambda: print("Search Triggered"))
        search_button.place(x=1140, y=5, height=30, width=100)
    
        
        
        Label(details_frame, text="Delivery Date:", font=("times new roman", 15), bg="white").grid(row=1, column=0, padx=10, pady=5)
        Delivery_Date_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Delivery_Date_entry.grid(row=1, column=1, padx=10, pady=5)
        Delivery_Date_entry=DateEntry(details_frame,width=18,font=("times new roman",15),state="readonly",date_pattern="dd/mm/yyyy")
        Bill_Date_date.grid(row=1,column=1,padx=10,pady=5)
        
        Label(details_frame, text="Shipping Address:", font=("times new roman", 15), bg="white").grid(row=1, column=2, padx=10, pady=5)
        Shipping_Address_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Shipping_Address_entry.grid(row=1, column=3, padx=10, pady=5)
        
        Label(details_frame, text="Status:", font=("times new roman", 15), bg="white").grid(row=1, column=4, padx=10, pady=5)
        
        Status_entry_combobox = ttk.Combobox(details_frame,
        values=('Confirmed','Replaced','Cancel'),font=('times new roman', 15),width=18,state='readonly')
        Status_entry_combobox.set('status')
        Status_entry_combobox.grid(row=1, column=5)
        Label(details_frame, text="From date:", font=("times new roman", 15), bg="white").grid(row=2, column=2, padx=3, pady=3)

        from_date_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        from_date_entry.grid(row=2, column=3, padx=3, pady=3)

        Label(details_frame, text="To date:", font=("times new roman", 15), bg="white").grid(row=2, column=4, padx=3, pady=3)

        to_date_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        to_date_entry.grid(row=2, column=5, padx=3, pady=3)
    
        from datetime import datetime
        
        # Insert current date
        today = datetime.now().strftime("%Y-%m-%d")
        from_date_entry.insert(0, today)
        to_date_entry.insert(0, today)
    
        Label(details_frame, text="Phone no:", font=("times new roman", 15), bg="white").grid(row=2, column=0, padx=10, pady=5)
        Phone_no_entry = Entry(details_frame, font=("times new roman", 15), bg="lightyellow")
        Phone_no_entry.grid(row=2, column=1, padx=10, pady=5)
        
        
        
        
        
       
        
        table_frame = Frame(billing_frame, bg="white")
        table_frame.place(x=20, y=193, width=1900, height=600)
    
        # Scrollbars
        scroll_y = ttk.Scrollbar(table_frame, orient=VERTICAL)
        scroll_y.pack(side=RIGHT, fill=Y)
    
        scroll_x = ttk.Scrollbar(table_frame, orient=HORIZONTAL)
        scroll_x.pack(side=BOTTOM, fill=X)
        
        # Create the Treeview
        global billing_table
        billing_table = ttk.Treeview(
            table_frame,
            columns=("select", "Bill_Number","DESCRIPTION", "RATING", "CAT_NO", "MAKE", "QUANTITY", "NET_QTY",
                     "LIST_PRICE", "DISCOUNT", "DISC_UNIT_PRICE", "DISC_GROSS_PRICE", "LP_GROSS_PRICE"),
            show='headings',
            yscrollcommand=scroll_y.set,
            xscrollcommand=scroll_x.set
        )
        
    
        billing_table.pack(side=LEFT, fill=BOTH, expand=1)
    
       
     
        # Configure scrollbars to control the treeview
        scroll_y.config(command=billing_table.yview)
        scroll_x.config(command=billing_table.xview)
        
        # Pack the treeview first
       
        
       
        
        
        # Headings and columns setup
        billing_table.heading("select", text="☐")  # for checkbox
        billing_table.heading("Bill_Number", text="BILLING_NO")
        billing_table.heading("DESCRIPTION", text="DESCRIPTION")
        billing_table.heading("RATING", text="RATING")
        billing_table.heading("CAT_NO", text="CAT NO.")
        billing_table.heading("MAKE", text="MAKE")
        billing_table.heading("QUANTITY", text="QUANTITY")
        billing_table.heading("NET_QTY", text="NET QTY")
        billing_table.heading("LIST_PRICE", text="LIST PRICE")
        billing_table.heading("DISCOUNT", text="DISCOUNT")
        billing_table.heading("DISC_UNIT_PRICE", text="DISC UNIT PRICE")
        billing_table.heading("DISC_GROSS_PRICE", text="DISC GROSS PRICE")
        billing_table.heading("LP_GROSS_PRICE", text="LP GROSS PRICE")
    
    
        billing_table.column("select", width=50, anchor=CENTER)
        billing_table.column("Bill_Number", width=100)
        billing_table.column("DESCRIPTION", width=150)
        billing_table.column("RATING", width=100)
        billing_table.column("CAT_NO", width=120)
        billing_table.column("MAKE", width=150)
        billing_table.column("QUANTITY", width=100, anchor=CENTER)
        billing_table.column("NET_QTY", width=100, anchor=CENTER)
        billing_table.column("LIST_PRICE", width=100, anchor=E)
        billing_table.column("DISCOUNT", width=100, anchor=E)
        billing_table.column("DISC_UNIT_PRICE", width=120, anchor=E)
        billing_table.column("DISC_GROSS_PRICE", width=120, anchor=E)
        billing_table.column("LP_GROSS_PRICE", width=120, anchor=E)
    
    
       
        
    
       
    
    
        
        # Total Frame
        total_frame = Frame(billing_frame, bg="white")
        total_frame.place(x=30, y=800, width=1240, height=50)
        
        total_label = Label(total_frame, text="Total Amount: ₹0.00", 
                           font=("times new roman", 14, "bold"), bg="white")
        total_label.pack(side=RIGHT, padx=20)
        
        def search_data():
            criterion = search_combo.get()
            query = search_entry.get().strip()
        
            if criterion == "Search By":
                messagebox.showwarning("Select Criterion", "Please select a search criterion.")
                return
            if not query:
                messagebox.showwarning("Input Error", "Please enter a search term.")
                return
        
            # Build SQL query dynamically (be careful with SQL injection - use parameters!)
            if criterion == "Bill No":
                sql ="""SELECT BillNo, Date,ClientID, ClientName, SupplierName, 
                                Status, Amount, Time 
                         FROM Billing 
                         WHERE BillNo = ?"""
                params = (query,)
    
            elif criterion == "Supplier name":
                sql ="""SELECT BillNo, Date, ClientID, ClientName, SupplierName, 
                                Status, Amount, Time 
                         FROM Billing 
                         WHERE SupplierName LIKE ?"""
                params = (f"%{query}%",)
            else:
                messagebox.showerror("Invalid Search", "Unsupported search criterion.")
                return
    
        
            param = query if criterion == "Invoice No" else f"%{query}%"
            cursor.execute(sql, param)
            rows = cursor.fetchall()
        
            # Clear existing rows
            billing_table.delete(*billing_table.get_children())
        
            # Insert query result into Treeview
            for row in rows:
                billing_table.insert("", "end", values=row)
    
    
        
    
            # Update total amount
            update_total_amount()
    
    
        
    
        def on_hover_billno(event):
            col = billing_table.identify_column(event.x)
            if col == "#2":  # Column for "Bill No"
                billing_table.config(cursor="hand2")
            else:
                billing_table.config(cursor="")
    
        def on_click(event):
            region = billing_table.identify("region", event.x, event.y)
            col = billing_table.identify_column(event.x)
            row_id = billing_table.identify_row(event.y)
    
            if region == "heading" and col == "#1":
                toggle_all_checkboxes(event)
            elif region == "cell":
                if col == "#1":
                    toggle_checkbox(event)
                elif col == "#2":  # "Bill No" column
                    open_invoice_detail(row_id)
    
    
        
    
    
    
    
    
    
        def toggle_checkbox(event):
            region = billing_table.identify("region", event.x, event.y)
            if region != "cell":
                return
    
            selected_item = billing_table.identify_row(event.y)
            selected_col = billing_table.identify_column(event.x)
        
            if selected_col == "#1":  # First column (checkbox column)
                values = list(billing_table.item(selected_item, "values"))
                values[0] = "✓" if values[0] == "☐" else "☐"
                billing_table.item(selected_item, values=values)
                
    
        select_all_flag = {"checked": False}  # Mutable flag for state toggle
    
        def toggle_all_checkboxes(event):
            col = billing_table.identify_column(event.x)
            if col == "#1":  # Header of 'select' column
                new_value = "✓" if not select_all_flag["checked"] else "☐"
                billing_table.heading("select", text=new_value)
                for item in billing_table.get_children():
                    values = list(billing_table.item(item, "values"))
                    values[0] = new_value
                    billing_table.item(item, values=values)
                select_all_flag["checked"] = not select_all_flag["checked"]
                
    
        
       
    
        def update_total():
            total = sum(float(billing_table.item(item, "values")[4]) for item in billing_table.get_children())
            total_label.config(text=f"Total Amount: ₹{total:.2f}")
    
        
       
    
        def save_invoice():
            if not billing_table.get_children():
                messagebox.showerror("Error", "No items to save!")
                return
    
            bill_no = Bill_entry.get()
            client_id = Client_entry.get()
            client_name = Client_Name_entry.get()
            supplier_name = Supplier_Name_entry.get()
            status = Status_entry_combobox.get()
            amount = amount_entry.get()
    
            if not (bill_no and client_id and client_name and supplier_name and status and amount):
                messagebox.showerror("Error", "Please fill in all bill details.")
                return
    
            try:
                conn = odbc.connect(connection_string)
                cursor = conn.cursor()
    
                total_amount = sum(float(billing_table.item(item, "values")[7]) for item in billing_table.get_children())
                bill_date = supplier_entry_date.get_date().strftime("%Y-%m-%d")
                bill_time = Time_entry.get()
    
                # ✅ Corrected: Match number of columns and values
                cursor.execute('''
                    INSERT INTO BILLING (BillNo, Date, ClientID,ClientName, SupplierName, Status, Amount, Time)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    bill_no,
                    bill_date,
                    client_id,
                    client_name,
                    supplier_name,
                    status,
                    float(amount),
                    bill_time
                ))
    
                for item in billing_table.get_children():
                    values = billing_table.item(item, "values")
                    cursor.execute('''
                        INSERT INTO BillingItems (BillNo, MaterialName, Quantity, Price, Amount)
                        VALUES (?, ?, ?, ?, ?)
                    ''', (bill_no, values[1], "", "", values[7]))  # Using placeholders for missing Quantity/Price
    
                conn.commit()
                messagebox.showinfo("Success", f"Bill {bill_no} saved successfully!")
                clear_all()
    
            except odbc.IntegrityError:
                messagebox.showerror("Error", "Bill number already exists.")
            except Exception as e:
                messagebox.showerror("Database Error", f"Error saving bill: {str(e)}")
            finally:
                if conn:
                    conn.close()
    
    
        def load_all_invoices():
            try:
                conn = odbc.connect(connection_string)
                cursor = conn.cursor()
        
                billing_table.delete(*billing_table.get_children())  # Clear table first
        
                # Example: Load from invoice_items table for latest invoice
                cursor.execute("""
                    SELECT TOP 50 BillNo, Date, CustomerID, CustomerName, 
                                 SupplierName, Status, Amount, Time 
                    FROM Billing 
                    ORDER BY BillNo DESC
                """)
    
        
                rows = cursor.fetchall()
                for row in rows:
                    billing_table.insert("", END, values=row)
        
                update_total()
                conn.close()
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load invoices:\n{str(e)}")
    
       
    
        def view_selected_invoices():
            selected = billing_table.selection()
            if not selected:
                messagebox.showwarning("No Selection", "Please select at least one invoice to view.")
                return
        
            for sel in selected:
                bill_no = billing_table.item(sel, "values")[1]  # bill_no column
        
                # --- Fetch from DB ---
                try:
                    conn = pyodbc.connect(connection_string)
                    cur = conn.cursor()
                    cur.execute("SELECT * FROM invoices WHERE bill_no = ?", bill_no)
                    inv = cur.fetchone()
                    if not inv:
                        messagebox.showerror("Error", f"No data for Bill No: {bill_no}")
                        return
        
                    cur.execute("""
                        SELECT m.material_name, ii.quantity, ii.list_price
                        FROM invoice_items ii
                        JOIN material m ON ii.material_id = m.material_id
                        WHERE ii.bill_no = ?
                    """, bill_no)
                    items = cur.fetchall()
                finally:
                    cur.close()
                    conn.close()
        
                # --- Build Window ---
                win = Toplevel()
                win.title(f"Invoice – {bill_no}")
                win.geometry("800x1000")
                win.config(bg="white")
        
                # Header bar
                header = Frame(win, bg="#f0f0f0", height=120)
                header.pack(fill=X)
                # Logo (replace "logo.png" with your path)
                try:
                    logo_img = Image.open("logo.png").resize((80,80))
                    logo = ImageTk.PhotoImage(logo_img)
                    Label(header, image=logo, bg="#f0f0f0").image = logo, \
                        Label(header, image=logo, bg="#f0f0f0").place(x=20,y=20)
                except:
                    pass
                Label(header, text="RUDRA ELECTRICAL INDUSTRIES",
                      font=("Helvetica", 20, "bold"),
                      bg="#f0f0f0").place(x=120, y=30)
                Label(header, text="ESTIMATION",
                      font=("Helvetica", 14),
                      bg="#f0f0f0").place(relx=0.85, y=50, anchor="center")
        
                # Top meta frame
                meta = Frame(win, bg="white")
                meta.pack(fill=X, padx=20, pady=10)
                # Left: Customer info
                left = Frame(meta, bg="white")
                left.pack(side=LEFT, fill=Y)
                Label(left, text=inv.customer_name, font=("Helvetica",14,"bold"), bg="white").pack(anchor="w")
                Label(left, text=f"PHONE NO : {inv.contact}", font=("Helvetica",11), bg="white").pack(anchor="w")
                Label(left, text=f"ADDRESS : {inv.shipping_address}", font=("Helvetica",11), bg="white").pack(anchor="w")
        
                # Right: Invoice title + quotation/date
                right = Frame(meta, bg="white")
                right.pack(side=RIGHT, fill=Y)
                Label(right, text="INVOICE", font=("Helvetica",20,"bold"), fg="#d35400", bg="white").pack(anchor="e")
                Label(right, text=f"QUOTATION NO #{inv.bill_no}", font=("Helvetica",11), bg="white").pack(anchor="e")
                Label(right, text=f"DATE : {inv.bill_date.strftime('%B %d, %Y')}",
                      font=("Helvetica",11), bg="white").pack(anchor="e")
        
                # Divider line
                Canvas(win, height=2, bg="#2c3e50").pack(fill=X, padx=20, pady=5)
        
                # Item table
                cols = ("Sr. No", "Material Description", "Quantity", "Rate/Unit", "Total")
                table_frame = Frame(win, bg="white")
                table_frame.pack(fill=BOTH, expand=True, padx=20)
                tree = ttk.Treeview(table_frame, columns=cols, show="headings", height=8)
                for c in cols:
                    tree.heading(c, text=c)
                    tree.column(c, anchor="center")
                tree.pack(side=LEFT, fill=BOTH, expand=True)
                Scrollbar(table_frame, orient=VERTICAL, command=tree.yview).pack(side=RIGHT, fill=Y)
                tree.configure(yscrollcommand=lambda *args: None)
        
                # Insert items
                subtotal = 0
                for i, (name, qty, rate) in enumerate(items, 1):
                    total = qty * float(rate)
                    subtotal += total
                    tree.insert("", END, values=(f"{i}.", name, qty, f"₹{rate:.2f}", f"₹{total:.2f}"))
        
                # Bottom divider
                Canvas(win, height=2, bg="#2c3e50").pack(fill=X, padx=20, pady=10)
        
                # Summary (right‑aligned)
                tax = subtotal * 0.02
                total_amt = subtotal + tax
                summ = Frame(win, bg="white")
                summ.pack(fill=X, padx=20)
                for lbl, val in [("Subtotal:", subtotal), (f"Tax (2%):", tax)]:
                    Label(summ, text=f"{lbl} ₹{val:,.2f}",
                          font=("Helvetica",11), bg="white").pack(anchor="e")
                Label(summ, text=f"TOTAL: ₹{total_amt:,.2f}",
                      font=("Helvetica",14,"bold"),
                      bg="white").pack(anchor="e", pady=(2,10))
                Canvas(summ, height=2, width=200, bg="#2c3e50").pack(anchor="e")
        
                # Terms & Conditions footer
                terms = Frame(win, bg="white")
                terms.pack(fill=BOTH, expand=True, padx=20, pady=20)
                Label(terms, text="TERMS AND CONDITIONS",
                      font=("Helvetica",12,"bold"), bg="white").pack(anchor="w", pady=(0,5))
                entries = [
                    ("Goods and Services", "18% extra"),
                    ("Prices", "Ex REI Works"),
                    ("Transportation", "Extra at actual"),
                    ("Payment Terms", "50% advance, Balance after material is ready."),
                    ("Validity offer", "7 days"),
                    ("Delivery", "8–10 working days after confirmation and advance"),
                    ("BOM", "As our standard BOM"),
                    ("SHEET CAL", "As our standard design")
                ]
                for left_txt, right_txt in entries:
                    row = Frame(terms, bg="white")
                    row.pack(fill=X, anchor="w")
                    Label(row, text=left_txt, font=("Helvetica",11), bg="white", width=20, anchor="w").pack(side=LEFT)
                    Label(row, text=right_txt, font=("Helvetica",11), bg="white", anchor="w").pack(side=LEFT)
        
                # PDF export
               
        
              
        
                win.mainloop()
        def generate_pdf():
                    from reportlab.pdfgen import canvas
                    from reportlab.lib.pagesizes import A4
                    from reportlab.lib.units import inch
        
                    path = filedialog.asksaveasfilename(defaultextension=".pdf",
                                                        filetypes=[("PDF files","*.pdf")])
                    if not path:
                        return
        
                    c = canvas.Canvas(path, pagesize=A4)
                    w, h = A4
                    y = h - inch
        
                    # Header
                    c.setFillColorRGB(0.95,0.95,0.95)
                    c.rect(0, y-80, w, 100, fill=1, stroke=0)
                    c.setFillColorRGB(0,0,0)
                    c.setFont("Helvetica-Bold", 18)
                    c.drawString(100, y-50, "RUDRA ELECTRICAL INDUSTRIES")
                    c.setFont("Helvetica", 12)
                    c.drawString(w-150, y-50, "ESTIMATION")
        
                    y -= 100
                    # Customer & Invoice meta
                    c.setFont("Helvetica", 10)
                    left_x, right_x = 50, w/2+20
                    meta_lines = [
                        inv.customer_name,
                        f"PHONE NO : {inv.contact}",
                        f"ADDRESS : {inv.shipping_address}"
                    ]
                    for i, line in enumerate(meta_lines):
                        c.drawString(left_x, y - i*12, line)
                    right_lines = [
                        "INVOICE",
                        f"QUOTATION NO #{inv.bill_no}",
                        f"DATE : {inv.bill_date.strftime('%B %d, %Y')}"
                    ]
                    c.setFont("Helvetica-Bold", 14)
                    c.drawString(right_x, y, right_lines[0])
                    c.setFont("Helvetica", 10)
                    for i, line in enumerate(right_lines[1:], 1):
                        c.drawString(right_x, y - i*12, line)
        
                    y -= 50
                    # Table header
                    c.setLineWidth(1)
                    c.line(40, y, w-40, y)
                    c.setFont("Helvetica-Bold", 10)
                    x_positions = [50, 120, 300, 380, 460]
                    for xi, text in zip(x_positions, cols):
                        c.drawString(xi, y-12, text)
                    y -= 25
                    c.line(40, y, w-40, y)
        
                    # Table rows
                    c.setFont("Helvetica", 9)
                    for idx, (name, qty, rate) in enumerate(items, 1):
                        total_r = qty * float(rate)
                        row_vals = [
                            f"{idx}.", name, str(qty),
                            f"₹{rate:.2f}", f"₹{total_r:.2f}"
                        ]
                        for xi, text in zip(x_positions, row_vals):
                            c.drawString(xi, y-12, text)
                        y -= 18
                        if y < 100:
                            c.showPage()
                            y = h - inch
        
                    # Summary
                    y -= 20
                    c.line(300, y, w-40, y)
                    y -= 18
                    c.drawRightString(w-50, y, f"Subtotal: ₹{subtotal:,.2f}")
                    y -= 14
                    c.drawRightString(w-50, y, f"Tax (2%): ₹{tax:,.2f}")
                    y -= 20
                    c.setFont("Helvetica-Bold", 12)
                    c.drawRightString(w-50, y, f"TOTAL: ₹{total_amt:,.2f}")
        
                    # Terms & Conditions
                    y -= 40
                    c.setFont("Helvetica-Bold", 10)
                    c.drawString(50, y, "TERMS AND CONDITIONS")
                    c.setFont("Helvetica", 9)
                    for i, (l, r) in enumerate(entries, 1):
                        c.drawString(50, y - i*12, l)
                        c.drawString(200, y - i*12, r)
        
                    c.save()
                    messagebox.showinfo("Saved", "Invoice PDF saved successfully.")
             
    
    
    
    
        
        
            # --- Container for total_frame content ---
            
        total_inputs_container = Frame(billing_frame, bg="white")
        total_inputs_container.place(x=30, y=800, width=1840, height=100) 
    
    # ----- ROW 0 -----
        row0 = Frame(total_inputs_container, bg="white")
        row0.pack(anchor="w", pady=2)
    
        Label(row0, text="ENCLOSURE FABRICATION:", font=("times new roman", 16), bg="white").pack(side=LEFT, padx=5)
        enclosure_fab_entry = Entry(row0, font=("times new roman", 16), bg="lightyellow", width=16)
        enclosure_fab_entry.pack(side=LEFT, padx=5)
    
        Label(row0, text="WIRING + ACCESSORIES:", font=("times new roman", 16), bg="white").pack(side=LEFT, padx=5)
        wiring_entry = Entry(row0, font=("times new roman", 16), bg="lightyellow", width=16)
        wiring_entry.pack(side=LEFT, padx=5)
    
        Label(row0, text="LABOUR:", font=("times new roman", 16), bg="white").pack(side=LEFT, padx=5)
        labour_entry = Entry(row0, font=("times new roman", 16), bg="lightyellow", width=16)
        labour_entry.pack(side=LEFT, padx=5)
    
    # ----- ROW 1 -----
        row1 = Frame(total_inputs_container, bg="white")
        row1.pack(anchor="w", pady=4)
    
        Label(row1, text="TRANSPORTATION:", font=("times new roman", 16), bg="white").pack(side=LEFT, padx=5)
        transportation_entry = Entry(row1, font=("times new roman", 16), bg="lightyellow", width=16)
        transportation_entry.pack(side=LEFT, padx=5)
    
        Label(row1, text="BUS BAR:", font=("times new roman", 16), bg="white").pack(side=LEFT, padx=5)
        bus_bar_entry = Entry(row1, font=("times new roman", 16), bg="lightyellow", width=16)
        bus_bar_entry.pack(side=LEFT, padx=5)
    
       
    
    
    # ----- Buttons (Right Side) -----

        
        
        # ----- ROW 2 for buttons -----
        row2 = Frame(total_inputs_container, bg="white")
        row2.pack(anchor="se", pady=0)

        Button(row2, text="Generate Bill", font=("times new roman", 20),
        bg="#4CAF50", fg="white", command=view_selected_invoices).pack(side=LEFT, padx=20)

        Button(row2, text="Print", font=("times new roman", 20),
        bg="#4CAF50", fg="white", command=generate_pdf).pack(side=LEFT, padx=20)

    
    
    
    
    
        # Connect buttons to functions
        
         # ✅ Now bind events after all handler functions are defined
        billing_table.bind("<Double-1>", toggle_checkbox)
        billing_table.bind("<Motion>", on_hover_billno)
        billing_table.bind("<ButtonRelease-1>", on_click) # end here
  


def payment_history():
    pass

# Quotation
quotation_btn = ctk.CTkButton(sidebar, text="Quotation", width=180, fg_color="#2C3E50", corner_radius=8)
quotation_btn.pack(pady=5)

# Invoices
invoices_btn = ctk.CTkButton(sidebar, text="Invoices", width=180, fg_color="#2C3E50", corner_radius=8)
invoices_btn.pack(pady=5)

# Order Status
order_status_btn = ctk.CTkButton(sidebar, text="Order status", width=180, fg_color="#2C3E50", corner_radius=8)
order_status_btn.pack(pady=5)

# Top Bar
topbar = ctk.CTkFrame(app, height=50, fg_color="#141C2B")
topbar.pack(fill="x", side="top")

search = ctk.CTkEntry(topbar, placeholder_text="Search products, customers, invoices...", width=600)
search.pack(padx=20, pady=10, side="left")

# Profile (right side)
ctk.CTkLabel(topbar, text="🧑 John Smith\nAdministrator", font=("Arial", 10), anchor="e").pack(side="right", padx=20)

# Main Content
main_frame = ctk.CTkFrame(app, fg_color="#0F172A")
main_frame.pack(fill="both", expand=True, padx=15, pady=10)

top_bar = ctk.CTkFrame(main_frame, fg_color="#1E293B")
top_bar.pack(fill="x", pady=(10, 5), padx=20)

welcome_label = ctk.CTkLabel(top_bar, text="👋 Welcome Admin", font=("Arial", 20, "bold"), text_color="#F8FAFC")
welcome_label.pack(side="left", padx=20)

datetime_label = ctk.CTkLabel(top_bar, font=("Arial", 16), text_color="#CBD5E1")
datetime_label.pack(side="right", padx=20)

def update_time():
    now = datetime.datetime.now()
    formatted = now.strftime("%A, %d %B %Y | %I:%M %p")
    datetime_label.configure(text=formatted)
    app.after(60000, update_time)  # update every 60,000 ms = 1 min

update_time()

# Cards (Revenue, Products, Customers, Orders)
info_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
info_frame.pack(fill="x", pady=(10, 0))

def open_revenue_window():
    app.lower()
    revenue_window = ctk.CTkToplevel(app)
    revenue_window.title("Total Revenue")
    revenue_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    revenue_window.state("zoomed")
    revenue_window.configure(fg_color="#0F172A")

    revenue_window.lift()
    revenue_window.attributes("-topmost", True)
    revenue_window.focus_force()
    revenue_window.after(500, lambda: revenue_window.attributes("-topmost", False))
    revenue_window.protocol("WM_DELETE_WINDOW", lambda: [revenue_window.destroy(), app.lift()])

    ctk.CTkLabel(revenue_window, text="💰 Total Revenue", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(revenue_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [revenue_window.destroy(), app.lift()]).pack(pady=10)

def open_products_window():
    app.lower()
    products_window = ctk.CTkToplevel(app)
    products_window.title("Total Products")
    products_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    products_window.state("zoomed")
    products_window.configure(fg_color="#0F172A")

    products_window.lift()
    products_window.attributes("-topmost", True)
    products_window.focus_force()
    products_window.after(500, lambda: products_window.attributes("-topmost", False))
    products_window.protocol("WM_DELETE_WINDOW", lambda: [products_window.destroy(), app.lift()])

    ctk.CTkLabel(products_window, text="📦 Total Products", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(products_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [products_window.destroy(), app.lift()]).pack(pady=10)

def fetch_employees():
    cursor.execute("""
        SELECT e.ID, e.Name, e.Designation, e.BasicSalary, e.PerDaySalary, e.Status,
               e.JoiningDate, e.ResigningDate, 
               s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted,
               (e.BasicSalary + (s.OTDayAmount * s.OTHours) - s.AdvanceDeducted) AS TotalSalary
        FROM Employees1 e
        JOIN EmployeeStat s ON e.ID = s.EmployeeID
    """)
    return cursor.fetchall()

# Add Employee

def add_employee_popup(parent, on_save_callback):
    popup = ctk.CTkToplevel(parent)
    popup.title("Add Employee")
    popup.geometry("400x500")
    popup.grab_set()

    # Title
    title_label = ctk.CTkLabel(popup, text="Add Employee", font=("Arial", 20, "bold"))
    title_label.pack(pady=10)

    # Name
    name_label = ctk.CTkLabel(popup, text="Name:")
    name_label.pack()
    name_entry = ctk.CTkEntry(popup)
    name_entry.pack(pady=5)

    # Designation
    designation_label = ctk.CTkLabel(popup, text="Designation:")
    designation_label.pack()
    designation_entry = ctk.CTkEntry(popup)
    designation_entry.pack(pady=5)

    # Basic Salary
    basic_salary_label = ctk.CTkLabel(popup, text="Basic Salary:")
    basic_salary_label.pack()
    basic_salary_entry = ctk.CTkEntry(popup)
    basic_salary_entry.pack(pady=5)

    per_day_salary_var = ctk.StringVar()

    per_day_salary_label = ctk.CTkLabel(popup, text="Per Day Salary:")
    per_day_salary_label.pack()
    per_day_salary_display = ctk.CTkEntry(popup, textvariable=per_day_salary_var)
    per_day_salary_display.configure(state="readonly")
    per_day_salary_display.pack(pady=5)

    # Calculate per-day salary as user types
    def update_per_day_salary(*args):
        try:
            basic = float(basic_salary_entry.get())
            per_day_salary = round(basic / 30, 2)
            per_day_salary_var.set(str(per_day_salary))
        except ValueError:
            per_day_salary_var.set("")

    basic_salary_entry.bind("<KeyRelease>", update_per_day_salary)

    # Status Dropdown
    status_label = ctk.CTkLabel(popup, text="Status:")
    status_label.pack()
    status_var = ctk.StringVar(value="Active")
    status_dropdown = ctk.CTkOptionMenu(popup, values=["Active", "Resigned"], variable=status_var)
    status_dropdown.pack(pady=5)

    # Joining Date (Date Picker)
    joining_label = ctk.CTkLabel(popup, text="Joining Date:")
    joining_label.pack()
    joining_entry = DateEntry(popup, width=18, background='darkblue', foreground='white', date_pattern='yyyy-mm-dd')
    joining_entry.pack(pady=5)

    # Save Button
    def save_employee():
        try:
            name = name_entry.get()
            designation = designation_entry.get()
            basic_salary = float(basic_salary_entry.get())
            per_day_salary = round(basic_salary / 30, 2)
            status = status_var.get()
            joining_date = joining_entry.get()

            # Insert into Employees1
            cursor.execute("""
                INSERT INTO Employees1 (Name, Designation, BasicSalary, Status, JoiningDate)
                VALUES (?, ?, ?, ?, ?)
            """, (name, designation, basic_salary, status, joining_date))
            conn.commit()

            # Get inserted employee ID
            cursor.execute("SELECT SCOPE_IDENTITY()")
            employee_id = cursor.fetchone()[0]

            print("Generated Employee ID:", employee_id)


            # Insert into EmployeeStats with 0 values
            cursor.execute("""
                INSERT INTO EmployeeStat (EmployeeID, OTDayAmount, OTHours, AdvanceTaken, AdvanceDeducted, TotalSalary)
                VALUES (?, 0, 0, 0, 0, 0)
            """, (employee_id,))
            conn.commit()
            def load_employees():
                cursor.execute("SELECT Name, Designation, BasicSalary, Status, JoiningDate FROM Employees1")
                rows = cursor.fetchall()
                for row in your_table_widget.get_children():
                    your_table_widget.delete(row)
                for row in rows:
                    your_table_widget.insert("", "end", values=row)


            messagebox.showinfo("Success", "Employee added successfully!")
            popup.destroy()
            on_save_callback()

        except ValueError:
            messagebox.showerror("Error", "Basic Salary must be a valid number.")
        except Exception as e:
            messagebox.showerror("Database Error", str(e))

    save_btn = ctk.CTkButton(popup, text="Save", command=save_employee)
    save_btn.pack(pady=20)

def refresh_employee_data(tree, selected_status):
    try:
        tree.delete(*tree.get_children())

        query = """
            SELECT e.ID, e.Name, e.Designation, e.BasicSalary, e.PerDaySalary, 
                   e.Status, e.JoiningDate, e.ResigningDate,
                   s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted, s.TotalSalary
            FROM Employees1 e
            JOIN EmployeeStat s ON e.ID = s.EmployeeID
        """
        if selected_status != "All":
            query += " WHERE e.Status = ?"
            cursor.execute(query, (selected_status,))
        else:
            cursor.execute(query)

        rows = cursor.fetchall()

        tree.tag_configure('evenrow', background='#f9f9f9')
        tree.tag_configure('oddrow', background='#e8e8e8')
        tree.tag_configure('Active', foreground='green')
        tree.tag_configure('resigned', foreground='red')

        for index, row in enumerate(rows):
            formatted = (
                str(row[0]), row[1], row[2],
                f"{row[3]:,.2f}", f"{row[4]:,.2f}",
                row[5], row[6], row[7],
                f"{row[8]:,.2f}", str(row[9]),
                f"{row[10]:,.2f}", f"{row[11]:,.2f}", f"{row[12]:,.2f}"
            )
            tag1 = 'evenrow' if index % 2 == 0 else 'oddrow'
            tag2 = 'Active' if row[5] == 'Active' else 'resigned'
            tree.insert('', 'end', values=formatted, tags=(tag1, tag2))

        return rows

    except Exception as e:
        messagebox.showerror("Error", str(e))
        return []


def search_employee(tree, keyword, selected_status):
    keyword = keyword.lower().strip()
    rows = refresh_employee_data(tree, selected_status)
    tree.delete(*tree.get_children())

    for index, row in enumerate(rows):
        if keyword in row[1].lower() or keyword in row[2].lower():
            formatted = (
                str(row[0]), row[1], row[2],
                f"{row[3]:,.2f}", f"{row[4]:,.2f}",
                row[5], row[6], row[7],
                f"{row[8]:,.2f}", str(row[9]),
                f"{row[10]:,.2f}", f"{row[11]:,.2f}", f"{row[12]:,.2f}"
            )
            tag1 = 'evenrow' if index % 2 == 0 else 'oddrow'
            tag2 = 'working' if row[5] == 'Working' else 'resigned'
            tree.insert('', 'end', values=formatted, tags=(tag1, tag2))


def sort_column(tv, col, reverse):
    data = [(tv.set(k, col), k) for k in tv.get_children('')]
    try:
        data.sort(key=lambda t: float(t[0].replace(",", "")), reverse=reverse)
    except:
        data.sort(reverse=reverse)
    for index, (_, k) in enumerate(data):
        tv.move(k, '', index)
    tv.heading(col, command=lambda: sort_column(tv, col, not reverse))


def open_employee_window(parent):
    emp_win = ctk.CTkToplevel(parent)
    emp_win.title("Employee Management")
    emp_win.state('zoomed')
    emp_win.focus_force()
    emp_win.grab_set()

    ctk.CTkLabel(emp_win, text="🧑‍💼 Employee Management", font=("Arial", 28, "bold")).pack(pady=10)

    # ==== Dashboard Cards ====
    card_frame = ctk.CTkFrame(emp_win)
    card_frame.pack(fill="x", padx=20, pady=10)

    def update_dashboard(frame):
        rows = refresh_employee_data(tree, status_var.get())
        total = len(rows)
        working = sum(1 for r in rows if r[5] == "Active")
        resigned = sum(1 for r in rows if r[5] == "Resigned")
        avg_salary = sum(float(r[3]) for r in rows) / total if total else 0

        metrics = [
            ("👥 Total", total, "#3498DB"),
            ("✅ Working", working, "#2ECC71"),
            ("❌ Resigned", resigned, "#E74C3C"),
            ("💰 Avg Salary", f"{avg_salary:,.2f}", "#F1C40F")
        ]
        for w in frame.winfo_children():
            w.destroy()

        for title, val, color in metrics:
            card = ctk.CTkFrame(frame, width=160, height=80, fg_color=color)
            card.pack_propagate(False)
            card.pack(side="left", padx=10)
            ctk.CTkLabel(card, text=title, font=("Arial", 11, "bold")).pack(pady=(8, 0))
            ctk.CTkLabel(card, text=str(val), font=("Arial", 20, "bold")).pack()

    # ==== Top Controls ====
    top_frame = ctk.CTkFrame(emp_win, fg_color="transparent")
    top_frame.pack(fill="x", padx=20, pady=10)

    status_var = ctk.StringVar(value="All")
    ctk.CTkLabel(top_frame, text="Status:", font=("Arial", 12)).pack(side="left", padx=5)
    status_menu = ctk.CTkOptionMenu(top_frame, variable=status_var,
                                    values=["All", "Active", "Resigned"],
                                    command=lambda _: [refresh_employee_data(tree, status_var.get()), update_dashboard(card_frame)])
    status_menu.pack(side="left", padx=10)

    ctk.CTkButton(top_frame, text="➕ Add Employee", 
        command=lambda: add_employee_popup(emp_win, lambda: [refresh_employee_data(tree, status_var.get()), update_dashboard(card_frame)])
                 ).pack(side="left", padx=10)

    search_var = ctk.StringVar()
    ctk.CTkEntry(top_frame, width=220, placeholder_text="Search by Name/Designation", textvariable=search_var).pack(side="right", padx=10)
    ctk.CTkButton(top_frame, text="🔍", command=lambda: search_employee(tree, search_var.get(), status_var.get())).pack(side="right")

    # ==== Treeview ====
    tree_frame = ctk.CTkFrame(emp_win)
    tree_frame.pack(fill="both", expand=True, padx=20, pady=10)

    columns = ["ID", "Name", "Designation", "BasicSalary", "PerDaySalary", "Status", 
               "JoiningDate", "ResigningDate", "OTDayAmount", "OTHours", 
               "AdvanceTaken", "AdvanceDeducted", "TotalSalary"]

    tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=30)
    for col in columns:
        tree.heading(col, text=col, command=lambda _col=col: sort_column(tree, _col, False))
        tree.column(col, anchor="center", width=110)

    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=tree.xview)
    tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

    vsb.pack(side="right", fill="y")
    hsb.pack(side="bottom", fill="x")
    tree.pack(fill="both", expand=True)

    refresh_employee_data(tree, status_var.get())
    update_dashboard(card_frame)


def open_orders_window():
    app.lower()
    orders_window = ctk.CTkToplevel(app)
    orders_window.title("Pending Orders")
    orders_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    orders_window.state("zoomed")
    orders_window.configure(fg_color="#0F172A")

    orders_window.lift()
    orders_window.attributes("-topmost", True)
    orders_window.focus_force()
    orders_window.after(500, lambda: orders_window.attributes("-topmost", False))
    orders_window.protocol("WM_DELETE_WINDOW", lambda: [orders_window.destroy(), app.lift()])

    ctk.CTkLabel(orders_window, text="🛒 Pending Orders", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(orders_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [orders_window.destroy(), app.lift()]).pack(pady=10)

def open_dispatch_window():
    app.lower()
    dispatch_window = ctk.CTkToplevel(app)
    dispatch_window.title("Logistics & Dispatch")
    dispatch_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    dispatch_window.state("zoomed")
    dispatch_window.configure(fg_color="#0F172A")

    dispatch_window.lift()
    dispatch_window.attributes("-topmost", True)
    dispatch_window.focus_force()
    dispatch_window.after(500, lambda: dispatch_window.attributes("-topmost", False))
    dispatch_window.protocol("WM_DELETE_WINDOW", lambda: [dispatch_window.destroy(), app.lift()])

    ctk.CTkLabel(dispatch_window, text="🚚 Logistics & Dispatch", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(dispatch_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [dispatch_window.destroy(), app.lift()]).pack(pady=10)
    
card_info = [
    ("💰 Total Revenue", "$124,532", "+12.5%", "#2563EB", open_revenue_window),
    ("📦 Total Products", "2,847", "+8.2%", "#22C55E", open_products_window),
    ("👥 Active Employees", "1,429", "+15.3%", "#9333EA", open_employee_window),
    ("🛒 Pending Orders", "23", "-4.2%", "#F97316", open_orders_window),
    ("🚚 Logistics & Dispatch", "5", "+2.1%", "#EF4444", open_dispatch_window)
]

for title, value, change, color, func in card_info:
    card = ctk.CTkFrame(info_frame, width=200, height=100, fg_color=color, cursor="hand2")
    card.pack_propagate(False)
    card.pack(side="left", padx=10, pady=10)

    label1 = ctk.CTkLabel(card, text=title, font=("Arial", 12, "bold"))
    label1.pack(anchor="w", padx=10, pady=(5, 0))

    label2 = ctk.CTkLabel(card, text=value, font=("Arial", 24, "bold"))
    label2.pack(anchor="w", padx=10)

    change_color = "green" if "+" in change else "red"
    label3 = ctk.CTkLabel(card, text=f"{change} vs last month", font=("Arial", 10), text_color=change_color)
    label3.pack(anchor="w", padx=10)

    # Bind cards with correct function signature
    def bind_click(widget, f=func):
        try:
            if f._code_.co_argcount == 1:
                widget.bind("<Button-1>", lambda e: f(app))
            else:
                widget.bind("<Button-1>", lambda e: f())
        except AttributeError:
            pass  # Skip if it's not a callable (guard)

    for widget in [card, label1, label2, label3]:
        bind_click(widget)


# Sales Overview
bottom_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
bottom_frame.pack(fill="both", expand=True)

sales_frame = ctk.CTkFrame(bottom_frame, fg_color="#1C1F2B")
sales_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)

ctk.CTkLabel(sales_frame, text="📊 Sales Overview", font=("Arial", 16, "bold")).pack(anchor="w", padx=10, pady=10)
months = [("Jan", 45), ("Feb", 52), ("Mar", 48), ("Apr", 61), ("May", 55), ("Jun", 67)]
for m, val in months:
    ctk.CTkLabel(sales_frame, text=f"{m} - ${val}k").pack(anchor="w", padx=20)
    bar = ctk.CTkProgressBar(sales_frame, width=200)
    bar.set(val / 100)
    bar.pack(padx=20, pady=2)

ctk.CTkLabel(sales_frame, text="Total Sales: $328,000", font=("Arial", 12, "bold")).pack(anchor="w", padx=20, pady=10)

# Inventory Status
inventory_frame = ctk.CTkFrame(bottom_frame, fg_color="#1C1F2B")
inventory_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)

ctk.CTkLabel(inventory_frame, text="📦 Inventory Status", font=("Arial", 16, "bold")).pack(anchor="w", padx=10, pady=10)

inv_data = [
    ("Smartphones", 145, 200, "#3498DB"),
    ("Laptops", 89, 120, "#2ECC71"),
    ("Tablets", 67, 80, "#9B59B6"),
    ("Accessories", 234, 300, "#E67E22"),
    ("Gaming", 45, 60, "#E74C3C")
]
for name, val, maxval, clr in inv_data:
    ctk.CTkLabel(inventory_frame, text=f"{name}: {val}/{maxval}", font=("Arial", 10)).pack(anchor="w", padx=20)
    prog = ctk.CTkProgressBar(inventory_frame, width=200, progress_color=clr)
    prog.set(val / maxval)
    prog.pack(padx=20, pady=3)

total_items = sum([val for _, val, _, _ in inv_data])
ctk.CTkLabel(inventory_frame, text=f"Total Items: {total_items}", font=("Arial", 12, "bold")).pack(anchor="w", padx=20, pady=10)

# Last updated
now = datetime.datetime.now().strftime("%d/%m/%Y, %I:%M:%S %p")
ctk.CTkLabel(main_frame, text=f"Last updated: {now}", font=("Arial", 10), text_color="#888").pack(anchor="e", padx=10, pady=5)

app.mainloop()
